In [2]:
#Importo as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import time
import openpyxl

In [3]:
def scoresfixtures(link,ids):
    '''
    Description: This Class picks all the games that had in one season and combinate all links to one especific list
    
    Inputs:
        - link: The link of the main page that have all season games desired.
        - ids: The ID of the championship table
        
    Outputs:
        - especific list that has all the links os all matches of the season
    
    
    '''
    req = requests.get(link)
    if req.status_code == 200:
        content = req.content

    soup = BeautifulSoup(content, 'html.parser')
    tb = soup.find(id=ids)
    
    s1= []
    s2= []
    for i in tb.find_all("a"):
            s1.append(str(i))
            s2.append(str(i.get_text('href')))


    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    di = pd.DataFrame(list(zip(s1, s2)), 
                   columns =['Codes', 'ID']) 

    s4=[]
    for i in di["Codes"]:
        i = i.replace('<a href="','')
        i = i.replace('</a>','')
        s4.append(str(i))


    s5 = []

    for i in di['Codes']:
        if "matches" in i:
            s5.append(str(i))
        else:
            s5.append(0)

    s6 = []
    for i in di["Codes"]:
        if '<a href="/en/squads/' in i:
            i = i.replace('<a href="/en/squads/','')
            i = i[0:8]
            s6.append(str(i))
        else:
            s6.append(0)        

    # Calling DataFrame constructor after zipping 
    # both lists, with columns specified 
    da = pd.DataFrame(list(zip(s1, s2,s4,s5,s6)), 
                   columns =['CODES', 'ID','URL_FINAL','PARTIDAS_2019',"TEAM_CODE"])        

    s9 = []
    for i in da["URL_FINAL"]:
        if 'Match Report' in i:
            s9.append(str(i))
        else:
            pass
    return s9


In [8]:
# 2021-2022 La Liga Scores & Fixtures

scoresfixtures("https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures","div_sched_11174_1")

['/en/matches/e69c743a/Valencia-Getafe-August-13-2021-La-Liga">Match Report',
 '/en/matches/7e4bbae4/Mallorca-Real-Betis-August-14-2021-La-Liga">Match Report',
 '/en/matches/bc5ea8ef/Cadiz-Levante-August-14-2021-La-Liga">Match Report',
 '/en/matches/fc0ee51a/Osasuna-Espanyol-August-14-2021-La-Liga">Match Report',
 '/en/matches/9613fb68/Alaves-Real-Madrid-August-14-2021-La-Liga">Match Report',
 '/en/matches/c1065902/Celta-Vigo-Atletico-Madrid-August-15-2021-La-Liga">Match Report',
 '/en/matches/23eddc5a/Barcelona-Real-Sociedad-August-15-2021-La-Liga">Match Report',
 '/en/matches/170fe01e/Sevilla-Rayo-Vallecano-August-15-2021-La-Liga">Match Report',
 '/en/matches/953c7040/Villarreal-Granada-August-16-2021-La-Liga">Match Report',
 '/en/matches/e803fa9b/Elche-Athletic-Club-August-16-2021-La-Liga">Match Report',
 '/en/matches/260fb527/Real-Betis-Cadiz-August-20-2021-La-Liga">Match Report',
 '/en/matches/b5a317ed/Alaves-Mallorca-August-21-2021-La-Liga">Match Report',
 '/en/matches/71c88c41/E

In [4]:
def spreadsheat(url):
    '''
    Description: This function goes to de URL of the match and treat all data in order to append it in one single Dataframe.
    
    Input:
        - url: Url of the html page
        
    Output:
        - Dataframe treated from the match saved on my machine excel file
    
    '''
    #make the request
    pg = 'https://fbref.com'
    url_pg = pg+ url
    req = requests.get(url_pg)
    if req.status_code == 200:
        content = req.content
    #accessing data from site
    soup = BeautifulSoup(content, 'html.parser')

    table_general = soup.find_all(class_ = "table_wrapper tabbed")
    table_time_1 = table_general[0]
    table_time_2 = table_general[1]
    table_time_3 = soup.find(class_='venuetime')

    #collecting data


    table_crowd = soup.find_all('div',class_ = "scorebox_meta")
    oi_crowd = str(table_crowd)


    #treating data
    toby = oi_crowd.split('<small>')
    crowd = str(toby[2])
    crowd = crowd.split('</small>')
    crowd = str(crowd[0])
    stadium = toby[4]
    stadium = stadium.split('</small>')
    stadium = str(stadium[0])    


    
    #collecting data
    data = table_time_3.get('data-venue-date')        

    
    #treating data
    name = str(soup.title)
    name = name.replace(" ","_")
    name = name.replace("<title>","")
    name = name.replace(".","")
    final_name = name.split("Report")[0]


    #treating data

    final_name = final_name.split("_Match")
    final_name = final_name[0]    


    # STR transform and reading tables
    table_str_1 = str(table_time_1)
    table_str_2 = str(table_time_2)
    df_1 = pd.read_html(table_str_1)[0]
    df_2 = pd.read_html(table_str_2)[0]

    #treating data

    time = str(final_name)
    time = time.replace("_"," ")
    time = time.split(" vs ")
    time_1 = str(time[0])
    time_2 = str(time[1])

    #Dtframe transforming
    df_1 = pd.DataFrame(df_1)
    df_1.columns = df_1.columns.droplevel()
    df_1['Team'] = str(time_1)
    df_1['Opponent_Team'] = str(time_2)
    df_1['Match'] = str(final_name)
    df_1['Date'] = str(data)
    df_1['Stadium'] = str(stadium)
    df_1['Crowd'] = crowd    


    df_2 = pd.DataFrame(df_2)
    df_2.columns = df_2.columns.droplevel()
    df_2['Team'] = str(time_2)
    df_2['Opponent_Team'] = str(time_1)
    df_2['Match'] = str(final_name)
    df_2['Date'] = str(data)
    df_2['Stadium'] = str(stadium)
    df_2['Crowd'] = crowd


    #APPENDING Dataframes
    df_3 = df_1.append(df_2)
    
    #save excel 
    writer = pd.ExcelWriter(final_name+'.xlsx')
    df_3.to_excel(writer,"Statistics")
    writer.save()

In [5]:
def treatment(name): 
    '''
    Description: This function goes through all files of the directiory and joins all them in one single dataframe saving in
    excel sheet.
    
    Input:
        - name: Name that you want for your excel sheet
        
    Output:
        - Dataframe of all games save as excel sheet
    
    
    '''
    entries = os.listdir()

    base = {}

    base = pd.DataFrame(base)


    for i in entries:
        if "xlsx" in i:
            base = base.append(pd.read_excel(i))
            base = base.drop_duplicates()

        #write in excelfile
        else:
            pass
    writer = pd.ExcelWriter("/home/nour/MalisProject/malis-project/season2021-2022"+name+ ".xlsx")
    base.to_excel(writer,name)
    writer.save()
    

In [18]:
start = time.time()
for url in scoresfixtures("https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures","div_sched_11174_1"):
    spreadsheat(url)
    

print('Duration: {} seconds'.format(time.time() - start))

Duration: 343.7330017089844 seconds


In [6]:
start = time.time()
treatment("Spain")

print('Duration: {} seconds'.format(time.time() - start))

Duration: 10.155680179595947 seconds
